In [25]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, make_scorer, average_precision_score, brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction import DictVectorizer
import joblib

ART = Path("artifacts")
RANDOM_SEED = 42


In [26]:
import json
with open(ART / "meta.json") as f:
    targets = json.load(f)["targets"]


In [27]:
def load_split(task, split):
    df = pd.read_csv(ART/f"{task}_{split}.csv")
    y = df[targets[task]]
    X = df.drop(columns=[targets[task]])
    return X, y


In [28]:
def to_records(X, task):
    # Prepare for DictVectorizer
    return [{c: r[c] for c in X.columns if pd.notna(r[c])} | {f"task={task}": 1} for _, r in X.iterrows()]


In [29]:
def tune_task(task):
    X, y = load_split(task, "train")
    vec = DictVectorizer(sparse=False)
    Xv = vec.fit_transform(to_records(X, task))
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    scorer = make_scorer(roc_auc_score, needs_proba=True)

    # Logistic Regression
    lr = LogisticRegression(max_iter=5000, class_weight="balanced")
    lr_grid = {"C": np.logspace(-3, 2, 15), "penalty": ["l2"], "solver": ["lbfgs", "liblinear"]}
    lr_rs = RandomizedSearchCV(lr, lr_grid, n_iter=12, scoring=scorer, cv=skf, random_state=RANDOM_SEED, n_jobs=-1).fit(Xv, y)

    # GradientBoostingClassifier
    gb = GradientBoostingClassifier()
    gb_grid = {"n_estimators": [100,200,300], "learning_rate": [0.03,0.1,0.2], "max_depth": [2,3,4], "subsample": [0.7,0.85,1.0]}
    gb_rs = RandomizedSearchCV(gb, gb_grid, n_iter=12, scoring=scorer, cv=skf, random_state=RANDOM_SEED, n_jobs=-1).fit(Xv, y)

    joblib.dump(vec, ART / f"{task}_tune_vectorizer.joblib")
    print(f"{task.upper()} best LR AUC={lr_rs.best_score_:.3f}, best params={lr_rs.best_params_}")
    print(f"{task.upper()} best GB AUC={gb_rs.best_score_:.3f}, best params={gb_rs.best_params_}")

    return {
        "task": task,
        "best": {
            "lr": {"params": lr_rs.best_params_, "auc": float(lr_rs.best_score_)},
            "gb": {"params": gb_rs.best_params_, "auc": float(gb_rs.best_score_)}
        }
    }


In [30]:
tasks = ["breast", "diabetes", "heart", "kidney"]
tuning_results = [tune_task(t) for t in tasks]
with open(ART / "tuning_results.json", "w") as f:
    json.dump(tuning_results, f, indent=2)


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


BREAST best LR AUC=nan, best params={'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(43.93970560760795)}
BREAST best GB AUC=nan, best params={'subsample': 0.7, 'n_estimators': 200, 'max_depth': 2, 'learning_rate': 0.1}


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


DIABETES best LR AUC=nan, best params={'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(43.93970560760795)}
DIABETES best GB AUC=nan, best params={'subsample': 0.7, 'n_estimators': 200, 'max_depth': 2, 'learning_rate': 0.1}


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


HEART best LR AUC=nan, best params={'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(43.93970560760795)}
HEART best GB AUC=nan, best params={'subsample': 0.7, 'n_estimators': 200, 'max_depth': 2, 'learning_rate': 0.1}


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


KIDNEY best LR AUC=nan, best params={'solver': 'liblinear', 'penalty': 'l2', 'C': np.float64(43.93970560760795)}
KIDNEY best GB AUC=nan, best params={'subsample': 0.7, 'n_estimators': 200, 'max_depth': 2, 'learning_rate': 0.1}


In [31]:
import json
with open(ART / "tuning_results.json") as f:
    tuning = {d["task"]: d["best"] for d in json.load(f)}


In [32]:
from sklearn.calibration import CalibratedClassifierCV

def train_final_model(task):
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier
    import joblib

    Xtr, ytr = load_split(task, "train")
    Xca, yca = load_split(task, "calib")
    Xte, yte = load_split(task, "test")
    vec = DictVectorizer(sparse=False)
    XTR = vec.fit_transform(to_records(Xtr, task))
    XCA = vec.transform(to_records(Xca, task))
    XTE = vec.transform(to_records(Xte, task))

    best_lr = tuning[task]["lr"]
    best_gb = tuning[task]["gb"]
    use_gb = best_gb["auc"] >= best_lr["auc"]
    if use_gb:
        model = GradientBoostingClassifier(**best_gb["params"])
    else:
        model = LogisticRegression(max_iter=5000, class_weight="balanced", **best_lr["params"])

    model.fit(XTR, ytr)
    cal = CalibratedClassifierCV(model, method="sigmoid", cv="prefit").fit(XCA, yca)
    joblib.dump(vec, ART/f"{task}_vectorizer.joblib")
    joblib.dump(cal, ART/f"{task}_calibrated_model.joblib")
    print(f"Saved {task} vectorizer and calibrated model.")
    return cal, vec, XTE, yte


In [33]:
for task in tasks:
    cal, vec, XTE, yte = train_final_model(task)
    p_te = cal.predict_proba(XTE)[:,1]
    print(f"\nResults for {task.upper()}:")
    print("AUROC:", roc_auc_score(yte, p_te))
    print("AUPRC:", average_precision_score(yte, p_te))
    print("Brier score:", brier_score_loss(yte, p_te))


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


Saved breast vectorizer and calibrated model.

Results for BREAST:
AUROC: 0.9636243386243386
AUPRC: 0.9455649565573653
Brier score: 0.07532242871944982


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


Saved diabetes vectorizer and calibrated model.

Results for DIABETES:
AUROC: 0.8229629629629629
AUPRC: 0.7016708705458802
Brier score: 0.16656441618101236
Saved heart vectorizer and calibrated model.

Results for HEART:
AUROC: 0.7832167832167832
AUPRC: 0.8373265623265623
Brier score: 0.2066517883864368
Saved kidney vectorizer and calibrated model.

Results for KIDNEY:
AUROC: 1.0
AUPRC: 1.0
Brier score: 0.02803798150797554


C:\Users\EXTECH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\calibration.py:330: FutureWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(


In [34]:
import math, numpy as np

def compute_risk_band_threshold(cal, XCA, yca, alpha=0.1):
    p_ca = cal.predict_proba(XCA)[:,1]
    p_true = np.where(yca==1, p_ca, 1 - p_ca)
    k = int(math.ceil((len(p_true)+1)*(1-alpha)))
    qhat = float(np.partition(1 - p_true, k-1)[k-1])
    return qhat

for task in tasks:
    Xtr, ytr = load_split(task, "train")
    Xca, yca = load_split(task, "calib")
    vec = joblib.load(ART/f"{task}_vectorizer.joblib")
    cal = joblib.load(ART/f"{task}_calibrated_model.joblib")
    XCA = vec.transform(to_records(Xca, task))
    qhat = compute_risk_band_threshold(cal, XCA, yca)
    with open(ART/f"{task}_conformal.json","w") as f:
        json.dump({"alpha":0.1,"qhat":qhat}, f)
    print(f"{task} risk band (90%): qhat={qhat:.3f}")


breast risk band (90%): qhat=0.412
diabetes risk band (90%): qhat=0.707
heart risk band (90%): qhat=0.611
kidney risk band (90%): qhat=0.158
